In [1]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange

from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import BertConfig, BertModel, BertTokenizer, BertForPreTraining

from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn import preprocessing

In [2]:
device = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'NUM_WORKERS':4,
    'EPITOPE_MAX_LEN':72,
    'EPOCHS':20,
    'LEARNING_RATE':5e-5,
    'BATCH_SIZE':1024,
    'THRESHOLD':0.5,   # 기본적으로 0.5로 사용하지만 data impalance가 심할 경우 더 큰 값을 사용하기도 한다.
    'SEED':41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
tokenizer = BertTokenizer.from_pretrained('model/tokenizer')

In [6]:
def seqtoinput(seq):
    for j in range(len(seq)-1):
        seq = seq[:j+j+1]+ ' ' + seq[j+j+1:]
    return seq
    
def get_protein_feature(seq, epitope):
    protein_feature = []
    protein_feature.append(ProteinAnalysis(seq).isoelectric_point())
    protein_feature.append(ProteinAnalysis(seq).aromaticity())
    protein_feature.append(ProteinAnalysis(seq).gravy())
    protein_feature.append(ProteinAnalysis(seq).instability_index())
    return protein_feature

def normalization(a):
    standard_scaler = preprocessing.StandardScaler()    
    standard_scaler.fit(a)
    result = standard_scaler.transform(a)
    return result
    
def get_preprocessing(data_type, new_df, tokenizer):
    epitope_ids_list = []
    epitope_mask_list = []
    
    protein_features = []
#     epitope_features = []
        
    for epitope, antigen, s_p, e_p in tqdm(zip(new_df['epitope_seq'], new_df['antigen_seq'], new_df['start_position'], new_df['end_position'])):             
        protein_features.append(get_protein_feature(antigen, epitope))
#         epitope_features.append(get_peptide_feature(epitope))
        
        epitope = seqtoinput(epitope)
        
        
        epitope_input = tokenizer(epitope, add_special_tokens=True, pad_to_max_length=True, max_length = 72)
        epitope_ids = epitope_input['input_ids']
        epitope_mask = epitope_input['attention_mask']
        
        
        epitope_ids_list.append(epitope_ids)
        epitope_mask_list.append(epitope_mask)

    
    label_list = None
    if data_type != 'test':
        label_list = []
        for label in new_df['label']:
            label_list.append(label)
    print(f'{data_type} dataframe preprocessing was done.')
    
    protein_features = normalization(protein_features)
    
    return epitope_ids_list, epitope_mask_list, protein_features, label_list

In [7]:
train = pd.read_csv('data/train.csv')
train = train.drop([7511])
test = pd.read_csv('data/test.csv')

train, val = train_test_split(train, train_size=0.8, random_state=12)

train_epitope_ids_list, train_epitope_mask_list, train_protein_features, train_label_list = get_preprocessing('train', train, tokenizer)
val_epitope_ids_list, val_epitope_mask_list, val_protein_features, val_label_list = get_preprocessing('val', val, tokenizer)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
152648it [04:14, 599.23it/s]


train dataframe preprocessing was done.


0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
38162it [01:03, 601.39it/s]

val dataframe preprocessing was done.


In [8]:
class CustomDataset(Dataset):
    def __init__(self, epitope_ids_list, epitope_mask_list, protein_features, label_list):
        self.epitope_ids_list = epitope_ids_list
        self.epitope_mask_list = epitope_mask_list
        self.protein_features = protein_features

        self.label_list = label_list
        
    def __getitem__(self, index):
        self.epitope_ids = self.epitope_ids_list[index]
        self.epitope_mask = self.epitope_mask_list[index]
        
        self.protein_feature = self.protein_features[index]        
        
        if self.label_list is not None:
            self.label = self.label_list[index]
            return torch.tensor(self.epitope_ids), torch.tensor(self.epitope_mask), torch.tensor(self.protein_feature, dtype=torch.float32), self.label
        else:
            return torch.tensor(self.epitope_ids), torch.tensor(self.epitope_mask), torch.tensor(self.protein_feature, dtype=torch.float32)
        
    def __len__(self):
        return len(self.epitope_ids_list)

In [9]:
train_dataset = CustomDataset(train_epitope_ids_list, train_epitope_mask_list, train_protein_features, train_label_list)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(val_epitope_ids_list, val_epitope_mask_list, val_protein_features, val_label_list)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [10]:
config = BertConfig(
    vocab_size=30, # default는 영어 기준이므로 내가 만든 vocab size에 맞게 수정해줘야 함
    hidden_size=768,
    num_hidden_layers=12,    # layer num
    num_attention_heads=12,    # transformer attention head number
    intermediate_size=3072,   # transformer 내에 있는 feed-forward network의 dimension size
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.0,
    max_position_embeddings=72,    # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정
    type_vocab_size=2,    # token type ids의 범위 (BERT는 segmentA, segmentB로 2종류)
)

In [11]:
pre = BertForPreTraining(config=config)
pre.save_pretrained('model/bert')

In [12]:
class TransformerModel(nn.Module):
    def __init__(self,
                 protein_hidden_dim=768,
                 pretrained_model='model/bert'
                ):
        super(TransformerModel, self).__init__()              
        # Transformer                
        self.transformer = BertModel.from_pretrained(pretrained_model)
                
        in_channels = protein_hidden_dim + 4
            
        self.classifier = nn.Sequential(
            nn.LeakyReLU(True),
            nn.BatchNorm1d(in_channels),
            nn.Linear(in_channels, protein_hidden_dim//4),
            nn.LeakyReLU(True),
            nn.BatchNorm1d(protein_hidden_dim//4),
            nn.Linear(protein_hidden_dim//4, 1)
        )
        
    def forward(self, epitope_ids_list, epitope_mask_list, protein_features):
        BATCH_SIZE = epitope_ids_list.size(0)
        # Get Embedding Vector
        epitope_x = self.transformer(input_ids=epitope_ids_list, attention_mask=epitope_mask_list)[0]
        
        # transformer [CLS]
        epitope_hidden = epitope_x[:, 0, :]
                        
        # Feature Concat -> Binary Classifier
        x = torch.cat([epitope_hidden, protein_features], axis=-1)
        x = self.classifier(x).view(-1)
        return x

In [13]:
class WeightedFocalLoss(nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self, alpha=.25, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha]).to(device)
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [14]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = WeightedFocalLoss().to(device) 
#     criterion = FocalLoss().to(device)
    best_val_f1 = 0
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for epitope_ids_list, epitope_mask_list, protein_features, label in tqdm(iter(train_loader)):
            epitope_ids_list = epitope_ids_list.to(device)
            epitope_mask_list = epitope_mask_list.to(device)

            protein_features = protein_features.to(device)

            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(epitope_ids_list, epitope_mask_list, protein_features)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
            if scheduler is not None:
                scheduler.step()
                    
        val_loss, val_f1, val_acc, val_f1_T, val_f1_F = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val F1 : [{val_f1:.5f}] Val acc : [{val_acc:.5f}] Val F1_T : [{val_f1_T:.5f}] Val F1_F : [{val_f1_F:.5f}] ')
        
        if best_val_f1 < val_f1:
            best_val_f1 = val_f1
#             torch.save(model.module.state_dict(), './feature_transformer_best_model.pth', _use_new_zipfile_serialization=False)
            torch.save(model.module.state_dict(), '../../../../../DAS_Storage4/daehun/epitope_best_model.pth', _use_new_zipfile_serialization=False)
            print('Model Saved.')
    return best_val_f1

In [15]:
from sklearn.metrics import accuracy_score

def validation(model, val_loader, criterion, device):
    model.eval()
    pred_proba_label = []
    true_label = []
    val_loss = []
    with torch.no_grad():
        for epitope_ids_list, epitope_mask_list, protein_features, label in tqdm(iter(val_loader)):
            epitope_ids_list = epitope_ids_list.to(device)
            epitope_mask_list = epitope_mask_list.to(device)

            protein_features = protein_features.to(device)
            
            label = label.float().to(device)
            
            model_pred = model(epitope_ids_list, epitope_mask_list, protein_features)
            loss = criterion(model_pred, label)
            model_pred = torch.sigmoid(model_pred).to('cpu')
            
            pred_proba_label += model_pred.tolist()
            true_label += label.to('cpu').tolist()
            
            val_loss.append(loss.item())
    
    pred_label = np.where(np.array(pred_proba_label)>CFG['THRESHOLD'], 1, 0)
    val_f1 = f1_score(true_label, pred_label, average='macro')
    f1_T = f1_score(true_label, pred_label)
    f1_F = f1_score(true_label, pred_label, pos_label=0)
    acc = accuracy_score(true_label, pred_label)
    return np.mean(val_loss), val_f1, acc, f1_T, f1_F

In [16]:
model = TransformerModel()
model = nn.DataParallel(model, device_ids=[3, 4, 5, 1])
model.eval()

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000*CFG['EPOCHS'], eta_min=0)

best_score = train(model, optimizer, train_loader, val_loader, scheduler, device)
print(f'Best Validation F1 Score : [{best_score:.5f}]')

Some weights of the model checkpoint at model/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 38/38 [00:29<00:00,  1.28it/s]


Epoch : [1] Train Loss : [0.04841] Val Loss : [0.03874] Val F1 : [0.68195] Val acc : [0.86777] Val F1_T : [0.43883] Val F1_F : [0.92506] 
Model Saved.


100%|██████████| 38/38 [00:30<00:00,  1.23it/s]

Epoch : [2] Train Loss : [0.04310] Val Loss : [0.04280] Val F1 : [0.64768] Val acc : [0.82417] Val F1_T : [0.39831] Val F1_F : [0.89704] 



100%|██████████| 38/38 [00:30<00:00,  1.25it/s]


...

Epoch : [48] Train Loss : [0.00225] Val Loss : [0.07244] Val F1 : [0.70384] Val acc : [0.91350] Val F1_T : [0.45465] Val F1_F : [0.95302] 



100%|██████████| 38/38 [00:28<00:00,  1.34it/s]


Epoch : [49] Train Loss : [0.00194] Val Loss : [0.07387] Val F1 : [0.70094] Val acc : [0.90823] Val F1_T : [0.45196] Val F1_F : [0.94992] 


100%|██████████| 38/38 [00:27<00:00,  1.36it/s]

Epoch : [50] Train Loss : [0.00215] Val Loss : [0.07264] Val F1 : [0.69280] Val acc : [0.90346] Val F1_T : [0.43841] Val F1_F : [0.94719] 
Best Validation F1 Score : [0.73656]


In [17]:
test_df = pd.read_csv('data/test.csv')
test_epitope_ids_list, test_epitope_mask_list, test_protein_features, test_label_list= get_preprocessing('test', test_df, tokenizer)

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
120944it [21:01, 95.87it/s] 


test dataframe preprocessing was done.


In [18]:
test_dataset = CustomDataset(test_epitope_ids_list, test_epitope_mask_list, test_protein_features, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [19]:
model = TransformerModel()
best_checkpoint = torch.load('../../../../../DAS_Storage4/daehun/epitope_best_model.pth', map_location=device)
model.load_state_dict(best_checkpoint)
model.eval()
model.to(device)

Some weights of the model checkpoint at model/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TransformerModel(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 768, padding_idx=0)
      (position_embeddings): Embedding(72, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [20]:
def inference(model, test_loader, device):
    model.eval()
    pred_proba_label = []
    with torch.no_grad():
        for epitope_ids_list, epitope_mask_list, protein_features in tqdm(iter(test_loader)):
            epitope_ids_list = epitope_ids_list.to(device)
            epitope_mask_list = epitope_mask_list.to(device)

            protein_features = protein_features.to(device)
            
            model_pred = model(epitope_ids_list, epitope_mask_list, protein_features)
            model_pred = torch.sigmoid(model_pred).to('cpu')
            
            pred_proba_label += model_pred.tolist()
    
    pred_label = np.where(np.array(pred_proba_label)>CFG['THRESHOLD'], 1, 0)
    return pred_label

In [21]:
preds = inference(model, test_loader, device)
submit = pd.read_csv('data/sample_submission.csv')
submit['label'] = preds

submit.to_csv('submission/epitope_submission.csv', index=False)
print('Done.')

100%|██████████| 119/119 [05:06<00:00,  2.58s/it]

Done.
